# Quantum Variational Monte Carlo Ising model in 1D using Stochastic Reconfiguration

In this notebook we will try to find the optimal parameters of a trial wavefunction describing the 1D quantum Ising model with varying number of spins. The trial wavefunction has an RBM-like expression:
$$
    \Psi(\mathcal{S}) = e^{b^Ts}\prod_{i=1}^{n_h}2\cosh(c_i + W^T_is)
$$
where $n_h$ is the number of hidden units, and $\mathbf{\theta} = \{\mathbf{b},\mathbf{c},\textbf{W}\}$ are our variational parameters. According to the variational method, this wavefunction will provide an upper bound to the ground state energy that we are trying to estimate:
$$
    E = \frac{\langle\Psi|\hat{H}|\Psi\rangle}{\langle\Psi|\Psi\rangle}
$$
where the Hamiltonian of the Transverse Field Ising model is:
$$
    \hat{H} = -J\sum_{i=1}^{N}\hat{\sigma}^z_i\hat{\sigma}^z_{i+1} -B\sum_{j=1}^N\hat{\sigma}^x_j
$$
with periodic boundary conditions: $\hat{\sigma}^z_L\hat{\sigma}^z_{L+1} \equiv \hat{\sigma}^z_L\hat{\sigma}^z_1$, and $\hat{\sigma}^z,\hat{\sigma}^x$ being the Pauli matrices acting on the spins, $J,B$ the coupling and the transverse magnetic field term, respectively.

## Statistical Analysis and Autocorrelation Time

Measurements in Markov chains are usually correlated to a certain degree. Though, correlation fades as the number of steps between measured states increases. The distance at which we can consider that two states are uncorrelated is called the **autocorrelation time** $\tau$. However, finding the value analytically is too difficult, so we rely in the binning analysis of the time series to estimate $\tau$.

### Binning Analysis

Assume that we do not have any prior knowledge about the autocorrelation time. Then, we have to use $N_B$ blocks of the time-series samples of increasing lengths $k=2^0,2^1,2^2,\ldots$ until the error estimate for the block converges:
$$
    \varepsilon \approx \sqrt{\frac{s^2_B}{N_B}},\quad\text{where}\quad s^2_B = \frac{1}{N_B-1}\sum_{i=1}^{N_B}(\langle f\rangle_{B_i} - \langle f\rangle_B)^2
$$
The $i$-th block average is computed as
$$
    \langle f\rangle_{B_i} = \frac{1}{k}\sum_{t=1}^kf(x_{(i-1)k+t})
$$
and the mean of all the block averages is
$$
    \langle f\rangle_B = \frac{1}{N_B}\sum_{i=1}\langle f\rangle_{B_i}
$$

The integrated autocorrelation time can be inferred from the binning analysis results. Being
$$
    \tau = \frac{1}{2}\frac{\frac{s^2_B}{N_B}(k\to\infty)}{\frac{s^2_B}{N_B}(k=1)}
$$

## Stochastic Reconfiguration

In order to find the optimal configuration of parameters which minimizes the energy, we will use stochastic reconfiguration of the gradients:
 - Starting from a random initial configuration of the parameters.
 - Proposing a new state $s\to s'$.
 - Accepting the change with probability $|\Psi(s')|^2/|\Psi(s)|^2$.
 - Updating the parameters in our wavefunction as follows:
$$
    \theta^{(t+1)}_k = \theta^{(t)}_k - \delta t\textbf{S}^{-1}\textbf{F}
$$
where we define $\textbf{S}$ as the Fubini-Study metric information matrix
$$
    S_{kk'} = \langle\Delta^*_k\Delta_{k'}\rangle - \langle\Delta^*_k\rangle\langle\Delta_{k'}\rangle
$$
to which we can add an extra regularization term to increase stability, by removing the null diagonal terms $S_{kk} \leftarrow S_{kk} + \lambda$. We also define $\Delta_k$ as the log-derivative of the k-th parameter, and $\textbf{F}$ as the gradient of the mean energy
$$
    F_k = \langle E_{\text{loc}}\Delta^*_k\rangle - \langle E_{\text{loc}}\rangle\langle\Delta^*_k\rangle
$$
To find the ground state of the Hamiltonian, we need to calculate the variational derivatives of the parameters in our RBM ansatz:
$$
    \Delta_{b_j}(s) = \frac{\partial}{\partial b_j}\log(\Psi(s)) = s_j
$$
$$
    \Delta_{c_i}(s) = \frac{\partial}{\partial c_i}\log(\Psi(s)) = \tanh(c_i + \sum_{j=1}^{N}\omega_{ij}s_j)
$$
$$
    \Delta_{\omega_{ij}}(s) = \frac{\partial}{\partial \omega_{ij}}\log(\Psi(s)) = s_j\tanh(c_i + \sum_{j=1}^{N}\omega_{ij}s_j)
$$

## Magnetization of the ground state

We can compare the (longitudinal/transversal) magnetization of the exact ground state with the mean of the sampled state found with Stochastic Reconfiguration. For that, we simply calculate
$$
    m_z = \sqrt{\langle(\hat{\sigma}^z)^2\rangle} = \sqrt{\langle\psi_0|(\tfrac{1}{N}\textstyle\sum_{i=1}^N\hat{\sigma}^z_i)^2|\psi_0\rangle}
$$
$$
    m_x = \sqrt{\langle(\hat{\sigma}^x)^2\rangle} = \sqrt{\langle\psi_0|(\tfrac{1}{N}\textstyle\sum_{i=1}^N\hat{\sigma}^x_i)^2|\psi_0\rangle}
$$
Another way of calculating the exact magnetization, would be to consider large $N$ (or $N\to\infty$), for which the magnetizations simply become:
$$
    m_z = \sqrt{\rho^z_N} = \sqrt{\langle\psi_0|\hat{\sigma}^z_i\hat{\sigma}^z_{i+N}|\psi_0\rangle} = \left(1 - \frac{B^2}{J^2}\right)^{1/8}
$$
if $B < J$, and $0$ otherwise. For the transversal magnetization, we have:
$$
    m_x = \frac{1}{\pi}\int_0^{\pi}\left(\frac{B + J\cos{k}}{\sqrt{J^2 + B^2 + 2JB\cos{k}}}\right)\text{d}k
$$
Now we would only need a way of comparing these values with our found solution with Stochastic Reconfiguration.

When the number of spins is small enough ($N\lesssim20$), we can calculate the projections of our RBM wavefunction onto the spin basis as
$$
    \braket{\mathcal{S}|\Psi_{\mathbf\theta}} = \begin{pmatrix} \Psi_{\mathbf\theta}(s_1) & \Psi_{\mathbf\theta}(s_2) & \ldots & \Psi_{\mathbf\theta}(s_{2^N-1}) & \Psi_{\mathbf\theta}(s_{2^N}) \end{pmatrix}^\intercal
$$

For higher number of spins, the probability density of the ground state can be approximated with a Monte Carlo sampling with the optimized parameters $\mathbf{\theta} = \{\mathbf{b},\mathbf{c},\textbf{W}\}$, given enough samples:
$$
    |\psi_0|^2\sim\frac{1}{N_{\text{samples}}}\sum_{i=1}^{N_{\text{samples}}}\langle\mathcal{S}|\Psi_{\theta}\rangle
$$
Which, can be used to approximate the longitudinal magnetization, since it is diagonal in our spin basis $\mathcal{S}$:
$$
    m_z = \sqrt{\psi'_0\cdot(\sigma^z)^2\cdot\psi_0} = \text{diag}(\sigma^z)\cdot|\psi_0|
$$
but not the transversal magnetization because it is not diagonal in the same spin basis.

### Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from RBM import RBM
import Ising1D as tfi
import SR_Learning as sr

# Set PATH to save txt files
PATH = ''

# Set PATH to save figures
FIG = ''

## Error plots vs N:

The experiment should be done multiple times to calculate the average and variance of the results.
After 10-20 iterations, the results can be plotted below.

If using a HPC cluster, writing a script that calls this file changing the REALIZATION number should be the most efficient approach.

In [ ]:
############### IMPORTANT #################
REALIZATION = 1
###########################################

np.random.seed(REALIZATION) # Lucky number

# List of n values
n_list = [2,4,8,16,20]

# List of nh values
h_list = [0.5,1,2,4]

# Hamiltonian terms
J, B = 2, 1

# Error arrays
energy_error = np.zeros((len(h_list),len(n_list)))
energy_error2 = np.zeros((len(h_list),len(n_list)))
energy_error3 = np.zeros((len(h_list),len(n_list)))
mz_error = np.zeros((len(h_list),len(n_list)))
mx_error = np.zeros((len(h_list),len(n_list)))
chix_error = np.zeros((len(h_list),len(n_list)))
chiz_error = np.zeros((len(h_list),len(n_list)))
distances = np.zeros((len(h_list),len(n_list)))
distances2 = np.zeros((len(h_list),len(n_list)))
distances3 = np.zeros((len(h_list),len(n_list)))

for (j,n) in enumerate(n_list):
    # Exact ground state and energy
    H = tfi.buildHamiltonian_sparse(n,J,B)
    energies, states = tfi.diagonalizeHamiltonian_sparse(H,T=3)
    
    # Calculate magnetizations
    Mz, Mx = tfi.buildMagnetizations_sparse(n)
    exact_mz = np.sqrt(states[:,0].conj().T @ Mz**2 @ states[:,0])
    exact_mx = np.sqrt(states[:,0].conj().T @ Mx**2 @ states[:,0])
    exact_chix = n*(states[:,0].conj().T @ Mx**2 @ states[:,0] - (states[:,0].conj().T @ Mx @ states[:,0])**2)
    exact_chiz = n*(states[:,0].conj().T @ Mz**2 @ states[:,0] - (states[:,0].conj().T @ Mz @ states[:,0])**2)
    for (i,h) in enumerate(h_list):
        # Initialising RBM
        n_visible, n_hidden = n, int(h*n)
        psi = RBM(n_visible, n_hidden, sigma=0.1)
    
        RBM_energy, RBM_mz, RBM_mx, RBM_chix, RBM_chiz = sr.stochastic_reconfiguration(psi,J,B)
    
        # Energy error
        energy_error[i,j] = np.abs((energies[0] - RBM_energy)/energies[0])
        energy_error2[i,j] = np.abs((energies[1] - RBM_energy)/energies[1])
        energy_error3[i,j] = np.abs((energies[2] - RBM_energy)/energies[2])
    
        # Magnetization errors
        mz_error[i,j] = np.abs((exact_mz - RBM_mz)/exact_mz)
        mx_error[i,j] = np.abs((exact_mx - RBM_mx)/exact_mx)
        chiz_error[i,j] = np.abs((exact_chiz - RBM_chiz)/exact_chiz)
        chix_error[i,j] = np.abs((exact_chix - RBM_chix)/exact_chix)
        
        # Fubini-Study metric
        psi_RBM = psi.wavefunction()
        distances[i,j] = tfi.FubiniStudy(states[:,0],psi_RBM)
        distances2[i,j] = tfi.FubiniStudy(states[:,1],psi_RBM)
        distances3[i,j] = tfi.FubiniStudy(states[:,2],psi_RBM)

In [ ]:
# Save results
np.savetxt(PATH + f"ground_energy_error_{REALIZATION}.txt",energy_error)
np.savetxt(PATH + f"first_energy_error_{REALIZATION}.txt",energy_error2)
np.savetxt(PATH + f"second_energy_error_{REALIZATION}.txt",energy_error3)
np.savetxt(PATH + f"ground_distances_{REALIZATION}.txt",distances)
np.savetxt(PATH + f"first_distances_{REALIZATION}.txt",distances2)
np.savetxt(PATH + f"second_distances_{REALIZATION}.txt",distances3)
np.savetxt(PATH + f"mz_error{REALIZATION}.txt",mz_error)
np.savetxt(PATH + f"mx_error{REALIZATION}.txt",mx_error)
np.savetxt(PATH + f"chiz_error{REALIZATION}.txt",chiz_error)
np.savetxt(PATH + f"chix_error{REALIZATION}.txt",chix_error)

### Average and variance results

In [ ]:
# List of n values
n_list = [2,4,8,16,20]


# List of nh values
h_list = [0.5,1,2,4]

# Hamiltonian terms
J, B = 2, 1

# Initialize magnetization arrays
energy_error = np.zeros((len(h_list),len(n_list)))
energy_error2 = np.zeros((len(h_list),len(n_list)))
energy_error3 = np.zeros((len(h_list),len(n_list)))
distances = np.zeros((len(h_list),len(n_list)))
distances2 = np.zeros((len(h_list),len(n_list)))
distances3 = np.zeros((len(h_list),len(n_list)))
mz_error = np.zeros((len(h_list),len(n_list)))
mx_error = np.zeros((len(h_list),len(n_list)))
chix_error = np.zeros((len(h_list),len(n_list)))

var_e0 = np.zeros((len(h_list),len(n_list)))
var_e1 = np.zeros((len(h_list),len(n_list)))
var_e2 = np.zeros((len(h_list),len(n_list)))
var_d0 = np.zeros((len(h_list),len(n_list)))
var_d1 = np.zeros((len(h_list),len(n_list)))
var_d2 = np.zeros((len(h_list),len(n_list)))
var_mz = np.zeros((len(h_list),len(n_list)))
var_mx = np.zeros((len(h_list),len(n_list)))
var_chix = np.zeros((len(h_list),len(n_list)))

# Total number of iterations done and saved
total = 20

for REALIZATION in range(1,total + 1):
    # Load data
    e0 = np.loadtxt(PATH + f"ground_energy_error_{REALIZATION}.txt")
    e1 = np.loadtxt(PATH + f"first_energy_error_{REALIZATION}.txt")
    e2 = np.loadtxt(PATH + f"second_energy_error_{REALIZATION}.txt")
    d0 = np.loadtxt(PATH + f"ground_distances_{REALIZATION}.txt")
    d1 = np.loadtxt(PATH + f"first_distances_{REALIZATION}.txt")
    d2 = np.loadtxt(PATH + f"second_distances_{REALIZATION}.txt")
    
    mz = np.loadtxt(PATH + f"mz_error_{REALIZATION}.txt")
    mx = np.loadtxt(PATH + f"mx_error_{REALIZATION}.txt")
    chix = np.loadtxt(PATH + f"chix_error_{REALIZATION}.txt")

    # Mean
    energy_error += e0/total
    energy_error2 += e1/total
    energy_error3 += e2/total
    distances += d0/total
    distances2 += d1/total
    distances3 += d2/total

    mz_error += mz/total
    mx_error += mx/total
    chix_error += chix/total    
    # Variances
    var_e0 += e0**2/total
    var_e1 += e1**2/total
    var_e2 += e2**2/total
    var_d0 += d0**2/total
    var_d1 += d1**2/total
    var_d2 += d2**2/total

    var_mz += mz**2/total
    var_mx += mx**2/total
    var_chix += chix**2/total

var_e0 -= energy_error**2
var_e1 -= energy_error2**2
var_e2 -= energy_error3**2
var_d0 -= distances**2
var_d1 -= distances2**2
var_d2 -= distances3**2

var_mz -= mz_error**2
var_mx -= mx_error**2
var_chix -= chix_error**2

## Plots

### Energy error plot

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, energy_error[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, energy_error[i,:], var_e0[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title('Relative Error of Ground State Energy of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_energy_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### Longitudinal magnetization error plot

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, mz_error[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, mz_error[i,:], var_mz[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title(r'Relative Error of Ground State $m_z$ of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_mz_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### Transversal magnetization error plot

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, mx_error[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, mx_error[i,:], var_mx[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title(r'Relative Error of Ground State $m_x$ of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_mx_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### Magnetic susceptibility error plot

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, chix_error[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, chix_error[i,:], var_chix[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title(r'Relative Error of magnetic susceptibility of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_chi_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### Distance error plot

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
for i in range(len(h_list)):
    #ax1.plot(n_list, distances[i,:], marker='o', linestyle='-', label=f'n_h={h_list[i]}')
    ax1.errorbar(n_list, distances[i,:], var_d0[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
ax2 = ax1.twinx()
y_min, y_max = ax1.get_ylim()
ax2.set_ylim(np.rad2deg(y_min), np.rad2deg(y_max))
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.title(r'Distance between RBM and exact state', fontsize=14)
ax1.set_xlabel('Number of spins (N)',fontsize=12)
ax1.tick_params('both', which='major', labelsize=12)
ax1.set_ylabel('Angle (rad)',fontsize=12)
ax2.tick_params('y', which='major', labelsize=12)
ax2.set_ylabel('Angle (deg)',fontsize=12)
ax1.legend(fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.7)
plt.savefig(FIG + 'RBM_ground_distance_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### First excited state errors

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, energy_error2[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, energy_error2[i,:], var_e1[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title('Relative Error of 1st Excited State Energy of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_first_energy_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
for i in range(len(h_list)):
    #ax1.plot(n_list, distances2[i,:], marker='o', linestyle='-', label=f'n_h={h_list[i]}')
    ax1.errorbar(n_list, distances2[i,:], var_d1[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
ax2 = ax1.twinx()
y_min, y_max = ax1.get_ylim()
ax2.set_ylim(np.rad2deg(y_min), np.rad2deg(y_max))
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.title(r'Distance between RBM and first excited state', fontsize=14)
ax1.set_xlabel('Number of spins (N)',fontsize=12)
ax1.tick_params('both', which='major', labelsize=12)
ax1.set_ylabel('Angle (rad)',fontsize=12)
ax2.tick_params('y', which='major', labelsize=12)
ax2.set_ylabel('Angle (deg)',fontsize=12)
ax1.legend(fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.7)
plt.savefig(FIG + 'RBM_first_excited_distance_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

### Second excited state errors

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(len(h_list)):
    plt.plot(n_list, energy_error3[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N')
    #plt.errorbar(n_list, energy_error3[i,:], var_e2[i,:], marker='o', linestyle='-', label=f'Nh={h_list[i]}N', capsize=5)
plt.grid(True, linestyle='--', alpha=0.7)
plt.semilogy()
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.yticks(fontsize=12)
plt.title('Relative Error of 2nd Excited State Energy of TFI Model', fontsize=14)
plt.xlabel('Number of Spins (N)', fontsize=12)
plt.ylabel('Relative Error', fontsize=12)
plt.legend(fontsize=12)
plt.savefig(FIG + 'error_second_energy_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(8,6))
for i in range(len(h_list)):
    #ax1.plot(n_list, distances3[i,:], marker='o', linestyle='-', label=f'n_h={h_list[i]}')
    ax1.errorbar(np.array(n_list), distances3[i,:], var_d2[i,:], fmt='-o', label=f'Nh={h_list[i]}N', capsize=5)
ax2 = ax1.twinx()
y_min, y_max = ax1.get_ylim()
ax2.set_ylim(np.rad2deg(y_min), np.rad2deg(y_max))
plt.xticks(np.arange(2,21,2), fontsize=12)
plt.title(r'Distance between RBM and second excited state', fontsize=14)
ax1.set_xlabel('Number of spins (N)',fontsize=12)
ax1.tick_params('both', which='major', labelsize=12)
ax1.set_ylabel('Angle (rad)',fontsize=12)
ax2.tick_params('y', which='major', labelsize=12)
ax2.set_ylabel('Angle (deg)',fontsize=12)
ax1.legend(fontsize=12)
ax1.grid(True, linestyle='--', alpha=0.7)
plt.savefig(FIG + 'RBM_second_excited_distance_vs_n.png', dpi=300, bbox_inches='tight')
plt.show()